In [160]:
import pandas as pd
import bs4 as bs
import requests
import logging
import spacy
import tagme
import json
import sys
import os
import re
nlp = spacy.load('en_core_web_sm')

# Write a given data to a JSON file in two mode, Write and Append

In [137]:
def writeLinksJson(article, append=False):
    file = 'dataBase/articles.json'
    if append:
        with open(file, 'a+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
    else:
        with open(file, 'w+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
            # ensure_ascii=False : pour bien afficher les caractéres accentueux

# Read from a JSON file if exist!.

In [138]:
def readLinksJson():
    file = 'dataBase/articles.json'
    if os.path.isfile(file):
        with open(file) as jsonIn:
            data = json.load(jsonIn)
            return data
    else:
        print("I can't find ", file)

# Accumulate a list of string elements to an string.

In [168]:
def listToString(list):
    res = " ".join(list)
    return res

# Accumulate all elements in the list to an string.

In [171]:
def accumulate(*args):
    """
    This function will get a list of strings as parameters and will return an accumulate string of all elements in the list. 
    """
    res = ""
    for sentence in args:
        res += sentence + " "
    return res

# Two different methods to return all common elements between two lists.

In [139]:
def common(a, b):
    c = [value for value in a if value in b]
    return c

def common2(a, b):
    return set(a).intersection(b)

# Remove all punctuations in a given string.

In [140]:
def removePunctuation(string):
    result = re.sub('[«»!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', string)
    return result

# Convert Italian veracity to its French equivalent.

In [141]:
def frenchVeracity(veracity):
    db = {'mark-vero': 'Vrai', 'mark-ceri': 'Quasi-Vrai', 'mark-ni': 'Faux',
          'mark-pinocchio': 'Presque-Faux', 'mark-panzana': 'Archi-Faux'}
    veracityResult = db.get(veracity, 'Faux')
    return veracityResult

# Create a list of all articles' links

In [142]:
def articleLinkIdGenerator(online=False):
    # on a utilisé le online=false car si on est pas en ligne on pourra
    # utiliser la dataBase pour afficher les resultats voulue
    url = 'https://pagellapolitica.it/'
    uriPage = 'dichiarazioni/verificato?page='
    jsonFile = 'dataBase/articles.json'
    articles = {}
    if online:
        for i in range(sys.maxsize):
            pageNumber = i
            pageNumber = str(pageNumber)
            pageAddress = url + uriPage + pageNumber

            resp = requests.get(pageAddress)
            if resp:
                source = requests.get(pageAddress).text
                soup = bs.BeautifulSoup(source, "lxml")
                tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
                for table in tables:
                    uriArticleID = table.find(
                        'a', {'class': 'statement-link'})['href']
                    idPattern = re.search(r'/([0-9]+)/', uriArticleID)
                    idNumber = idPattern.group(1)
                    articles[idNumber] = {
                        'url': url+uriArticleID,
                    }
                if idNumber == str(215):
                    break
            if idNumber == str(215):
                break

        with open(jsonFile, 'w') as file:
            # la sortie de Json est une variable ici c'est a
            json.dump(articles, file, indent=2, ensure_ascii=False)
    else:
        with open(jsonFile) as file:
            articles = json.load(file)
    return articles

# Extraction all data from an article by its ID

In [143]:
def extractInfoArticle(id, online=False):

    article = {}
    url = 'https://pagellapolitica.it/'
    if online:
        uriArticle = 'dichiarazioni/'
        address = url + uriArticle + id
        #print(address)
        resp = requests.get(address)
        if resp:
            source = requests.get(address).text
            soup = bs.BeautifulSoup(source, "lxml")

            # auteur
            p = soup.find(
                'p', {'class': 'h4 mb-1 px-2 text-dark font-weight-light'})
            author = p.find('a', {'class': 'u-link-muted'}).text.strip()

            # le titre au complet quand on rentre dans l'article
            divFullBody = soup.find('div', {'class': 'col-lg-9 mb-9 mb-lg-0'})
            if divFullBody:
                # veracity
                # h2 mark-ni
                # h2 mark-vero
                # h2 mark-ceri
                # h2 mark-pinocchio
                # h2 mark-panzana

                veracities = ["mark-ni", "mark-vero",
                              "mark-ceri", "mark-pinocchio", "mark-panzana"]
                divTitle = divFullBody.find('div', {'class': 'mb-2 mt-2 px-2'})
                divTitlePH2Px2 = divFullBody.find(
                    'p', {'class': 'h2 px-2'})
                if divTitle:
                    fullTitle = divTitle.find(
                        'span', {'class': 'h2'}).text.strip()

                    spanVeracity = divTitle.find(
                        'span', {'class': 'h2'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]
                elif divTitlePH2Px2:
                    fullTitle = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'}).text.strip()
                    spanVeracity = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]

                else:
                    # dans les anciens artictles le fullText n'existe pas donc
                    # si il ne le trouve pas il affiche le msg
                    fullTitle = "No fullTitle"
                    veracity = "No veracity"

            # les claims
            pclaimPX3 = soup.find(
                'p', {'class': 'lead px-3 py-3 bg-light g-brd-around g-brd-lightblue'})
            pclaimPX2 = soup.find('p', {'class': 'h2 px-2'})
            if pclaimPX3:
                claim = pclaimPX3.text.strip()
            elif pclaimPX2:
                claim = pclaimPX2.find(
                    'span', {'class': 'text-darker'}).text.strip()

            # les dates(publication,origine) elles sont dans la méme classe d'ou l'utilisation
            # de divDates[0] pour la publication et divDates[1] pour l'origine
            divDate = soup.find('div', {'class': 'card-body pt-0 px-2'})
            divDates = divDate.findAll('div', {'class': 'col-lg-2 text-left'})
            datePublished = divDates[0].find(
                'span', {'class': 'text-dark'}).text.strip()
            dateOrigin = divDates[1].find(
                'span', {'class': 'text-dark'}).text.strip()

            # statement source
            divReferredLinks = divDate.find('div', {'class': 'col-lg-4'})
            statementSource = divReferredLinks.find(
                'a', {'class': 'u-link-muted'})['href']

            # le join est pour concaténé les textes des balises P pour les mettre en un seul txt
            divMainArticle = soup.find('div', {'id': 'analisi-content'})
            listOfPInMainArticle = divMainArticle.findAll('p')
            mainText = [text.text.strip() for text in listOfPInMainArticle]
            mainArticleWithPunctuaion = ' '.join(mainText)
            mainArticle = removePunctuation(mainArticleWithPunctuaion)


            # les liens de reference dans le main text
            listOfAllLinks = []
            for link in divMainArticle.findAll('a', href=True):
                #print(link['href'])
                listOfAllLinks.append(link)
                #if link['href']:
                    #print("link:",link['href'])
            listOfAllLinksHref = [link['href'] for link in listOfAllLinks if link['href']]

            # Tags
            if divFullBody.find('div', {'class': 'px-2 u-space-2-top'}):
                divTags = divFullBody.find(
                    'div', {'class': 'px-2 u-space-2-top'})
                tags = divTags.findAll(
                    'a', {'class': 'btn btn-sm btn-light u-btn-light transition-3d-hover rounded-0 mb-2'})

                tagList = []
                for tag in tags:
                    tagList.append(tag.text.strip())

    else:
        data = readLinksJson()
        # pour l'ajout d'un element dans un dictionnaire exsitant ,**dic[2] recupére
        # tout les anciens elements du dic
        data[id] = {
            **data[id],
            # le site de fact checking
            "source": "source not found",
            "claim": "claim not found",
            "body": "mainArticle not found",                     # le text de l'article
                    "referred_links": "listOfAllLinksHref not found",    # tous les liens dans le texte
                    "title": "fullTitle not found",                      # le titre de l'article
                    "date": "dateOrigin not found",                      # date de la claim
                    "url": "address not found",  # url de l'article
                    "tags": "tags not found",                          # les mots cles
                    "author": "author not found",                       # auteur de la claim
                    "datePublished": "datePublished not found",
                    "rating_value": "rating_value not found",          # la valeur de la veracite
                    "statementSource": "statementSource not found",
                    # les entities nomes qui est extraite de la claim
                    "claim_entities": "claim_entities not found",
                    # les entities nomes qui est extraite de l'article
                    "body_entities": "body_entities not found",
                    # parmi les tages, les entities nomes a partir de la tag
                    "keyword_entities": "keyword_entities not found",
                    # les entities nomes a partir de l'auteur de la claim
                    "author_entities": "author_entities not found",
                    "review_author": "review_author not found"         # l'auteur de l'article

        }

    article = {
        id: {
            "source": url,                           # le site de fact checking
            "claim": claim,
            "body": mainArticle,                     # le text de l'article
            "referred_links": listOfAllLinksHref,    # tous les liens dans le texte
            "title": fullTitle,                      # le titre de l'article
            "date": dateOrigin,                      # date de la claim
            "url": address,  # url de l'article
            "tags": tagList,                          # les mots cles
            "author": author,                       # auteur de la claim
            "datePublished": datePublished,
            "rating_value": veracity,          # la valeur de la veracite
            "statementSource": statementSource,
            # les entities nomes qui est extraite de la claim
            "claim_entities": "claim_entities not found",
            # les entities nomes qui est extraite de l'article
            "body_entities": "body_entities not found",
            # parmi les tages, les entities nomes a partir de la tag
            "keyword_entities": "keyword_entities not found",
            # les entities nomes a partir de l'auteur de la claim
            "author_entities": "author_entities not found",
            "review_author": "review_author not found"         # l'auteur de l'article

        }
    }

    return article

In [22]:
article = extractInfoArticle("1327", online=True)
for link in article["1327"]["referred_links"]:
    print(link)

http://www.quirinale.it/qrnw/statico/costituzione/costituzione.htm#Principi
http://www.salute.gov.it/ministero/sezMinistero.jsp?label=ssn
http://www.regione.piemonte.it/pianosanitario/dwd/piano_socio_sanitario.pdf
http://www.salute.gov.it/portale/news/p3_2_1_1_1.jsp?lingua=italiano&menu=notizie&p=dalministero&id=936
http://www.parlamento.it/parlam/leggi/deleghe/98112dl.htm


# Get a list of article's links by page number

In [155]:
def pageLinks(pageNumber):
    url = 'https://pagellapolitica.it/'
    uri = 'dichiarazioni/verificato?page='
    pageAddress = url + uri + pageNumber
    resp = requests.get(pageAddress)
    if resp:
        source = requests.get(pageAddress).text
        soup = bs.BeautifulSoup(source, "lxml")
        tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
        pageLinks = []
        for table in tables:
            link = table.find('a', {'class': 'statement-link'})['href']
            articleLink = url+link
            pageLinks.append(articleLink)
    return pageLinks

In [24]:
pageLinks = pageLinks("1")
for i in pageLinks:
    print(i)

https://pagellapolitica.it/dichiarazioni/8595/si-il-mes-non-puo-prendere-decisioni-senza-il-voto-dellitalia
https://pagellapolitica.it/dichiarazioni/8594/quali-promesse-non-sono-state-mantenute-per-il-ponte-di-genova
https://pagellapolitica.it/dichiarazioni/8593/si-litalia-ha-gia-un-extra-deficit-da-80-miliardi-di-euro
https://pagellapolitica.it/dichiarazioni/8592/salvini-ha-ragione-sui-furbi-che-non-rispettano-la-quarantena
https://pagellapolitica.it/dichiarazioni/8591/no-non-sono-gia-disponibili-540-miliardi-di-euro-grazie-allue
https://pagellapolitica.it/dichiarazioni/8590/no-non-riaprono-i-musei-mentre-le-messe-rimangono-vietate
https://pagellapolitica.it/dichiarazioni/8589/no-lemilia-romagna-non-ha-il-primato-di-morti-per-covid-19-nelle-rsa
https://pagellapolitica.it/dichiarazioni/8588/di-maio-sbaglia-sul-funzionamento-di-immuni-la-app-di-contact-tracing
https://pagellapolitica.it/dichiarazioni/8587/debito-perpetuo-e-a-tasso-zero-e-davvero-possibile
https://pagellapolitica.it/dich

# Extract id of each article by ReGex

In [49]:
def idNumber(link):
    idPattern = re.search(r'/([0-9]+)/', link)
    idNumber = idPattern.group(1)
    return idNumber

# Testing extractInfoArticle() and idNumber()

At first execute the method pageLinks(pageNumber)

In [145]:
#for pageNumber in range(42, 174):
pageLinks = pageLinks("82")
for page in pageLinks:
    id = idNumber(page)
    lastArticle = extractInfoArticle(id, online=True)
    #ter.writeLinksJson(lastArticle, append=True)
    #print('claim = ', lastArticle[id]["body"])
    #print('URL = ', lastArticle[id]["url"])
    #print('rating_value = ', lastArticle[id]["rating_value"])
    #print("\n")

# A method to find the page number of an ID. This is for debugging purpose.

At first execute the method pageLinks(pageNumber)

In [38]:
def findPageNumberOfID(id):
    for pageNumber in range(1, 200):
        print("Looking in page", pageNumber)
        pageURLs = pageLinks(str(pageNumber))
        for page in pageURLs:
            idWanted = idNumber(page)
            if idWanted == id:
                print("ID", id, "is in the page:", pageNumber)
                break
        #if idWanted == id:
            #break

In [39]:
findPageNumberOfID("4473")

Looking in page 1
ID 8613 is in the page: 1
Looking in page 2
Looking in page 3
Looking in page 4
Looking in page 5
Looking in page 6
Looking in page 7
Looking in page 8
Looking in page 9
Looking in page 10
Looking in page 11
Looking in page 12
Looking in page 13
Looking in page 14
Looking in page 15
Looking in page 16
Looking in page 17
Looking in page 18
Looking in page 19
Looking in page 20
Looking in page 21
Looking in page 22
Looking in page 23
Looking in page 24
Looking in page 25
Looking in page 26
Looking in page 27
Looking in page 28
Looking in page 29
Looking in page 30
Looking in page 31
Looking in page 32
Looking in page 33
Looking in page 34
Looking in page 35
Looking in page 36
Looking in page 37
Looking in page 38
Looking in page 39
Looking in page 40
Looking in page 41
Looking in page 42
Looking in page 43
Looking in page 44
Looking in page 45
Looking in page 46
Looking in page 47
Looking in page 48
Looking in page 49
Looking in page 50
Looking in page 51
Looking in pag

# Create a DataFrame from a list of IDs. This is for debugging purpose.

In [53]:
def dataFrameOfIDs(articlesIDs):
    data = {}
    for id in articlesIDs:
        currentArticle = extractInfoArticle(id, online=True)
        data.update(currentArticle)
    df = pd.DataFrame(data).T   # Use .T or transpose() to swap rows and columns
    return df

In [54]:
articlesIDs = ["4473","8548","8547","8546"]
df = dataFrameOfIDs(articlesIDs)
#df.to_csv("pagellapolitica.csv")
df

,source,claim,body,referred_links,title,date,url,tags,author,datePublished,rating_value,statementSource,claim_entities,body_entities,keyword_entities,author_entities,review_author
4473,https://pagellapolitica.it/,"""L'Italia ha un avanzo primario di circa 40 mi...",Beppe Grillo parla di conti pubblici e di debi...,[http://www.mef.gov.it/doc-finanza-pubblica/de...,"""L'Italia ha un avanzo primario di circa 40 mi...",22 ott 2014,https://pagellapolitica.it/dichiarazioni/4473,"[C'eri quasi, Economia, M5S, PIL]",Beppe Grillo,06 nov 2014,mark-ceri,http://www.beppegrillo.it/2014/10/fuoridalleur...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8548,https://pagellapolitica.it/,«La Germania diede 300 miliardi per salvare le...,"L’11 marzo, in un’intervista al Corriere della...",[https://www.corriere.it/politica/20_marzo_11/...,"No, nel 2008 la Germania non ha dato 300 milia...",11 mar 2020,https://pagellapolitica.it/dichiarazioni/8548,"[Germania, Lega, Banche, Coronavirus]",Matteo Salvini,13 mar 2020,mark-ni,https://www.corriere.it/politica/20_marzo_11/c...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8547,https://pagellapolitica.it/,«I moduli di autosegnalazione dei pugliesi rie...,"Il 9 marzo mattina, ospite di Agorà su Rai3, i...",[https://twitter.com/agorarai/status/123692837...,Emiliano impreciso sui pugliesi militari e su ...,09 mar 2020,https://pagellapolitica.it/dichiarazioni/8547,"[Michele Emiliano, Militari, Coronavirus, pugl...",Michele Emiliano,10 mar 2020,mark-ni,https://twitter.com/agorarai/status/1236928377...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8546,https://pagellapolitica.it/,«Con i nostri governi eravamo ad ospitare 50 m...,"Il 9 marzo, in un’intervista a La Repubblica, ...",[https://rep.repubblica.it/ws/detail/intervist...,Renzi esagera sui detenuti nelle carceri itali...,10 mar 2020,https://pagellapolitica.it/dichiarazioni/8546,"[Giustizia, Carceri, Italia Viva]",Matteo Renzi,10 mar 2020,mark-ni,https://rep.repubblica.it/ws/detail/intervista...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found


# Convert JSON to Pandas DataFrame

In [130]:
with open("pagellapolitica.json") as jsonFile:
    #df = json.load(jsonFile)
    df = pd.read_json(jsonFile).T

In [127]:
df.head()

,source,claim,body,referred_links,title,date,url,tags,author,datePublished,rating_value,statementSource,claim_entities,body_entities,keyword_entities,author_entities,review_author
8614,https://pagellapolitica.it/,«Noi dal 1980 al 2018 abbiamo speso in media 1...,L’11 maggio in un’intervista con l’Huffington ...,[http://www.governo.it/it/i-governi-dal-1943-a...,Gli interessi sul debito ci sono costati 185 m...,11 mag 2020,https://pagellapolitica.it/dichiarazioni/8614,"[Debito Pubblico, Lega, Interessi]",Giancarlo Pagliarini,13 mag 2020,Vrai,https://www.huffingtonpost.it/entry/viva-il-me...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8613,https://pagellapolitica.it/,«Nel 2020 non avremo i 61 milioni di turisti c...,Il 12 maggio la deputata di Forza Italia Sandr...,[https://www.facebook.com/SandraSavinoTrieste/...,L’Italia rischia di perdere oltre 60 milioni d...,12 mag 2020,https://pagellapolitica.it/dichiarazioni/8613,"[Turismo, Forza Italia, Coronavirus]",Sandra Savino,13 mag 2020,Vrai,https://www.facebook.com/SandraSavinoTrieste/p...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8612,https://pagellapolitica.it/,«In tutti i Paesi europei le scuole riaprono e...,Il 12 maggio il leader della Lega Matteo Salvi...,[https://twitter.com/matteosalvinimi/status/12...,Salvini sbaglia dicendo che «in tutti i Paesi ...,12 mag 2020,https://pagellapolitica.it/dichiarazioni/8612,"[Coronavirus, Scuole, scuole chiuse, riapertur...",Matteo Salvini,12 mag 2020,Faux,https://twitter.com/matteosalvinimi/status/125...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8611,https://pagellapolitica.it/,«Il governo sta lavorando [...] a una maxi san...,Il 5 maggio l’ex ministro dell’Interno Matteo ...,[https://www.facebook.com/watch/live/?v=189703...,"No, la regolarizzazione dei migranti non sareb...",05 mag 2020,https://pagellapolitica.it/dichiarazioni/8611,"[migranti, sanatoria]",Matteo Salvini,12 mag 2020,Faux,https://www.facebook.com/salviniofficial/video...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found
8610,https://pagellapolitica.it/,«Chi continua a parlare di assenza di “condizi...,Il 9 maggio la deputata del M5s Dalila Nesci h...,[https://www.facebook.com/nescidalila/posts/29...,Il Mes avrà condizioni? Molto difficile (ma no...,09 mag 2020,https://pagellapolitica.it/dichiarazioni/8610,"[Mes, pandemic crisis support, trojka, troika]",Dalila Nesci,12 mag 2020,Faux,https://www.facebook.com/nescidalila/posts/290...,claim_entities not found,body_entities not found,keyword_entities not found,author_entities not found,review_author not found


# Convert Pandas DataFrame to CSV

In [128]:
%pwd

'/Users/su6i/Amir/gitProjects/myGitProjects/S2/TER/TER/database'

In [131]:
df.to_csv("pagellapolitica.csv", encoding='utf-8', header=True)

# spaCy NLP module

At first execute the method pageLinks(pageNumber)

In [156]:
pageLinks = pageLinks("2")
for page in pageLinks:
    id = idNumber(page)
    lastArticle = extractInfoArticle(id, online=True)
    text = lastArticle[id]["body"]
    print("Article ID:",id)
    print("*"*50)
    print('text = ', text,"\n")
    print("*"*50," spaCy ","*"*50)
    doc = nlp(text) 
    for ent in doc.ents: 
        print(ent.text, ent.start_char, ent.end_char, ent.label_) 


Article ID: 8598
**************************************************
text =  Aggiornamento 12 maggio 2020 Una precedente versione di questo factchecking conteneva alcune imprecisioni relative al testo di legge approvato dal Parlamento svedese per dare maggiori poteri al governo come abbiamo spiegato qui Il 3 maggio ospite a Non è l’Arena su La7 la leader di Fratelli d’Italia Giorgia Meloni è tornata min 1108 sulla decisione di fine marzo 2020 del Parlamento ungherese che ha assegnato poteri straordinari al primo ministro ungherese Viktor Orbán All’epoca Meloni aveva già difeso Orbán esagerando però – come avevamo scritto – nel paragonare la situazione ungherese con quella esistente in Italia in particolare per via dei decreti del presidente del Consiglio dei ministri Dpcm firmati da Giuseppe Conte Questa volta la leader di Fratelli d’Italia ha detto al conduttore Massimo Giletti che non l’ha fatto solo Orbán in Europa di farsi dare “pieni poteri” dal Parlamento ma l’ha fatto per esempio

Article ID: 8597
**************************************************
text =  Il 30 aprile ospite a Piazza Pulita su La7 l’ex segretario del Partito democratico Pier Luigi Bersani ha detto min 048 che in Italia il 46 per cento dei contribuenti dichiara meno di 15 mila euro mentre solo il 6 per cento ne dichiara più di 50 mila Questi dati li leggono anche in Germania e in Olanda e ricavano una semplice conclusione che gli italiani non vogliono tirare fuori i soldi che hanno ha aggiunto l’ex ministro delle Infrastrutture commentando le trattative in sede europea per affrontare l’emergenza coronavirus con i due Paesi citati che sono tra i più diffidenti nell’aiutare altri Stati in crisi come Italia e Spagna Ma i numeri citati da Bersani sono giusti o no Abbiamo verificato Quanto dichiarano i contribuenti in Italia Il 23 aprile il Dipartimento delle Finanze del Ministero dell’Economia ha pubblicato i dati sulle dichiarazioni dei redditi Irpef – la principale tassa sul reddito nel nostro Paes

Article ID: 8596
**************************************************
text =  Il 4 maggio in un’intervista con La Stampa la ministra dell’Interno Luciana Lamorgese ha commentato l’inizio della Fase 2 dell’emergenza coronavirus dicendo che il Ministero dell’Interno ha dato indicazioni ai prefetti sulla definizione di “congiunti” ribadendo l’interpretazione della presidenza del Consiglio dei ministri Ma è davvero così Abbiamo verificato Che cosa ha detto Palazzo Chigi Il Dpcm Il 26 aprile il presidente del Consiglio Giuseppe Conte ha annunciato in conferenza stampa l’inizio della cosiddetta “Fase 2” nella gestione dell’epidemia per il 4 maggio anticipando una serie di novità previste dal nuovo decreto della presidenza del Consiglio dei ministri Dpcm Tra le altre cose il decreto stabilisce che con la Fase 2 sono consentiti solo gli spostamenti motivati da comprovate esigenze lavorative o situazioni di necessità oppure per motivi di salute e che si considerano necessari gli spostamenti per i

Article ID: 8594
**************************************************
text =  Il 28 aprile l’ex ministro delle Infrastrutture Danilo Toninelli M5s ha commentato sul Blog delle Stelle il completamento della struttura del nuovo Ponte di Genova a un anno e otto mesi circa dal crollo del viadotto autostradale avvenuto il 14 agosto 2018 Secondo Toninelli il ponte l’abbiamo fatto pagare a chi lo doveva gestire per assenza di manutenzione e l’ha fatto crollare e i tempi sono stati rispettati L’ex ministro ha ragione o no Abbiamo verificato Le promesse sui tempi della ricostruzione Partiamo dalla questione sulle tempistiche della ricostruzione del ponte Già a febbraio 2019 Pagella Politica aveva verificato come non fossero state mantenute alcune delle promesse fatte dalle autorità sulle tempistiche per la demolizione di quanto era rimasto in piedi del viadotto Il quotidiano di Genova Il Secolo XIX ha invece raccolto quasi 100 promesse sulla ricostruzione fatte dal 14 agosto 2018 al 22 aprile 202

Article ID: 8593
**************************************************
text =  Il 28 aprile in un’intervista con Il Sole 24 Ore il ministro dello Sviluppo economico Stefano Patuanelli Movimento 5 stelle ha detto che tra il precedente decreto e quello in arrivo siamo già a un extra deficit di 80 miliardi I due decreti a cui fa riferimento Patuanelli sono il “Cura Italia” convertito definitivamente in legge dal Parlamento il 24 aprile mentre con quello in arrivo indica il cosiddetto “decreto Aprile” la cui approvazione da parte del Consiglio dei ministri è stata annunciata per il prossimo 30 aprile Ma il ministro Patuanelli cita una cifra corretta o no Abbiamo verificato Def e deficit Quando si parla di “deficit” si fa riferimento a quello che in gergo economico è chiamato anche “indebitamento netto” una statistica che indica quanto lo Stato spende di più di quanto incassa ogni anno tenendo conto anche del costo degli interessi sul debito pubblico che nel caso italiano sono molto consistent

...

Article ID: 8590
**************************************************
text =  Il 26 aprile la ministra per le Pari opportunità e la Famiglia Elena Bonetti Italia Viva ha commentato la conferenza stampa con cui il presidente del Consiglio Giuseppe Conte ha annunciato le nuove misure della cosiddetta “Fase 2” che entreranno in vigore dal prossimo 4 maggio In sicurezza si potrà visitare un museo ma non si può celebrare una funzione religiosa Questa decisione è incomprensibile ha scritto su Twitter Bonetti che ha espresso una critica simile anche su Facebook dove ha sottolineato che abbiamo già deciso di riaprire i musei Ma le cose stanno davvero così È vero che con le nuove regole riapriranno i musei mentre le chiese resteranno chiuse Abbiamo verificato Che cosa dice il nuovo Dpcm Musei e luoghi di cultura Il 26 aprile Conte ha firmato un nuovo decreto del presidente del Consiglio dei ministri Dpcm con una serie di misure che saranno valide dal prossimo 4 maggio al 17 maggio Il comma j dell

Article ID: 8588
**************************************************
text =  Il 22 aprile in un’intervista con SkyTg24 il ministro degli Esteri Luigi Di Maio M5s ha commentato min 019 lo scopo di Immuni l’applicazione di contact tracing che le autorità italiane impiegheranno per tracciare i contatti delle persone contagiate da nuovo coronavirus nel nostro Paese durante la cosiddetta “Fase 2” dell’epidemia Secondo l’ex capo politico del M5s Immuni serve a permettere a un cittadino di avere una segnalazione nel caso in cui stia per entrare a contatto con un positivo Abbiamo verificato e Di Maio fa un po’ di confusione Che cos’è Immuni e come funzionerà Il 16 aprile scorso il commissario straordinario per l’emergenza Domenico Arcuri ha firmato un’ordinanza con cui ha comunicato che un gruppo di esperti nominato dal governo ha selezionato la applicazione Immuni – sviluppata dalla società milanese Bending Spoons – per essere utilizzata nelle prossime settimane nel tracciamento dei contatti c

Article ID: 8586
**************************************************
text =  Il 22 aprile in un’intervista a La Repubblica l’ex presidente del Consiglio Enrico Letta Partito democratico ha commentato l’attuale emergenza coronavirus dicendo che la Lombardia è diventata l’epicentro mondiale dellepidemia da Covid19 mentre il resto del Paese ha cifre tedesche o danesi I numeri ufficiali gli danno ragione o no Abbiamo verificato I dati sul contagio in Italia Secondo i dati più aggiornati della Protezione civile dall’inizio dell’epidemia al 22 aprile ore 17 i casi registrati di nuovo coronavirus in Italia erano in totale 187327 così suddivisi 69092 in Lombardia 369 per cento e 118235 nel resto del Paese 631 per cento Le morti registrate erano invece 25085 12740 in Lombardia 508 per cento e 12345 492 per cento in tutte le altre regioni Ricordiamo che la Lombardia è la regione più popolosa d’Italia con 10 milioni e 61 mila abitanti su circa 60 milioni e 400 mila In base ai dati ufficiali – che 

Article ID: 8585
**************************************************
text =  Il 20 aprile l’ex presidente del Consiglio e attuale commissario europeo agli Affari economici Paolo Gentiloni Partito democratico ha commentato su Twitter il forte calo del prezzo del petrolio di questi giorni scrivendo che 45 anni fa di fronte alla crisi del petrolio la Comunità europea fece un’emissione di bond comunitari Il riferimento di Gentiloni è al dibattito delle ultime settimane dove alcuni Paesi europei – come Italia Francia e Spagna – si sono schierati a favore della creazione di obbligazioni a livello europeo chiamate anche “coronabond” o “eurobond” per fare fronte all’emergenza coronavirus Questa ipotesi ha però finora incontrato le opposizioni di altri Stati come Germania e Paesi Bassi Ma è vero come dice Gentiloni che già 45 anni fa furono emessi titoli di Stato comunitari Abbiamo verificato Crisi del petrolio ed Europa negli anni Settanta Alla fine del 1973 i consumi mondiali di petrolio in cr

Article ID: 8584
**************************************************
text =  Il 20 aprile nella sua Enews della settimana il leader di Italia Viva Matteo Renzi ha scritto che i grandi Paesi europei si dividono tra chi riapre le scuole ad aprile e chi le riapre a maggio In Italia non si sa ancora se le scuole riapriranno entro la fine dell’anno scolastico Il 17 aprile in un’intervista al Corriere della Sera la ministra dell’Istruzione Lucia Azzolina ha però detto che con l’attuale situazione sanitaria ogni giorno che passa allontana la possibilità di riaprire a maggio Ma è vero invece come sostiene Renzi che nei grandi Paesi europei – in Germania Francia Spagna e Regno Unito – gli studenti torneranno sui banchi tra aprile e maggio Abbiamo verificato e l’ex presidente del Consiglio sbaglia Che cosa hanno deciso per ora gli altri Paesi Ue Spagna Partiamo dalla Spagna Qui secondo i dati Unesco circa 8 milioni di studenti – dalla preprimaria all’università – sono obbligati a rimanere a casa 

19 3 5 CARDINAL
Fatto Quotidiano 47 63 PERSON
l’ex 64 68 GPE
Movimento 82 91 PERSON
5 92 93 CARDINAL
Alessandro Di Battista 101 123 ORG
l’affidabilità 134 148 ORG
Paese 200 205 NORP
30 280 282 CARDINAL
al 478 480 ORG
Quando 543 549 GPE
30 652 654 CARDINAL
Abbiamo 734 741 ORG
La 784 786 ORG
Fondo 837 842 ORG
Fmi 868 871 PERSON
al Pil 939 945 PERSON
115 950 953 CARDINAL
1990 974 978 DATE
Abbiamo 986 993 ORG
30 1111 1113 CARDINAL
Stati 1160 1165 ORG
al Pil Al 1361 1370 PERSON
Norvegia 61 1402 1413 PERSON
Qatar 588 1436 1445 GPE
Repubblica 1464 1474 GPE
Congo 1479 1484 GPE
44 1485 1487 CARDINAL
Prima 1498 1503 GPE
Paese 1515 1520 GPE
Emirati Arabi Uniti 1537 1556 PERSON
426 1557 1560 CARDINAL
l’Azerbaijan 42 1571 1586 PRODUCT
l’Angola 286 1597 1609 GPE
Nuova Zelanda 1623 1636 PERSON
283 1637 1640 CARDINAL
Filippine 1654 1663 ORG
259 1664 1667 CARDINAL
246 1688 1691 CARDINAL
Russia 1707 1713 GPE
178 1714 1717 CARDINAL
Paese 1738 1743 GPE
Paesi 1779 1784 NORP
Germania 1798 1806 ORG
Francia 1

# Tagme NLP module

In [172]:
tagme.GCUBE_TOKEN = "7d611b53-1694-4c90-8b18-8ba26a30c03b-843339462"
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
language = "it"

This function will get an string as a parameter and will return a list of all found entities.

In [179]:
def wikiLinks(text, language="it"):
    """
    This function will get an string as a parameter and will return a list of all found entities.
    """
    listOfLinks = []
    obj = Annotation_mentions(text)
    for namedIdentity in obj.keys():
        norm_title = tagme.normalize_title(namedIdentity)
        wikiTitle = tagme.wiki_title(norm_title)
        url = tagme.title_to_uri(wikiTitle, lang=language)
        listOfLinks.append(url)
    return listOfLinks

Tagme method to discover the concepts of wiki entities in those texts

In [178]:
def Annotation_mentions(txt):
    """
    Discover the concepts of wiki concept entities in those texts
    :param txt: a text object, str type
    :return: key-value pair, the key is the original entity concept in this article, the value is the concept size of the concept as a wiki concept, and those that belong to the wiki concept but have ambiguity also include
    """
    annotation_mentions = tagme.mentions(txt)
    dic = dict()
    for mention in annotation_mentions.mentions:
        try:
            dic[str(mention).split(" [")[0]] = str(mention).split("] lp=")[1]
        except:
            logger.error('error annotation_mention about ' + mention)
    return dic

In [173]:
url = 'https://pagellapolitica.it/dichiarazioni/8553/salvini-chiede-di-riaprire-il-parlamento-dove-ha-il-10-per-cento-di-presenze'
print(url)
id = idNumber(url)
data = extractInfoArticle(id, online=True)
body = data[id]['body']
author = data[id]['author']
claim = data[id]['claim']
tags = data[id]['tags']
tagList = listToString(tags)
print("author:",author,"\nclaim:",claim,"\ntagList:",tagList)
language = "it"

#for tag in tags:
acc_data = accumulate(body,author,claim,tagList)
print("="*115)
print(acc_data)

https://pagellapolitica.it/dichiarazioni/8553/salvini-chiede-di-riaprire-il-parlamento-dove-ha-il-10-per-cento-di-presenze
author: Rosalba Cimino 
claim: «Il senatore Salvini dall’alto del suo 10% di presenze in Senato, chiede di aprire il Parlamento che non si è mai fermato» 
tagList: Senato salvini Camera dei deputati Assenze in Senato
La deputata del M5s Rosalba Cimino il 22 marzo ha criticato sulla sua pagina Facebook le richieste di Matteo Salvini di riaprire il Parlamento sottolineando due elementi che il Parlamento non si è mai fermato durante l’emergenza coronavirus e che Salvini in ogni caso è stato presente appena nel 10 per cento dei casi nel corso della legislatura Andiamo a verificare se è vero Il Parlamento italiano ai tempi della Covid19 Il Parlamento italiano è in effetti aperto come testimoniano i resoconti dei lavori sia del Senato sia della Camera ma si sta riunendo con una frequenza minore rispetto al solito A marzo 2019 fino al 23 incluso ad esempio l’Assemblea del

In [180]:
if __name__ == '__main__':
    #f = open("text3.txt", "r", encoding="utf8")
    #txt = f.read()
    
    links = wikiLinks(acc_data)
    for link in links:
        print(link)

https://it.wikipedia.org/wiki/M5s
https://it.wikipedia.org/wiki/Rosalba
https://it.wikipedia.org/wiki/Cimino
https://it.wikipedia.org/wiki/Il_22
https://it.wikipedia.org/wiki/Marzo
https://it.wikipedia.org/wiki/Sulla
https://it.wikipedia.org/wiki/Sua
https://it.wikipedia.org/wiki/Facebook
https://it.wikipedia.org/wiki/Di_Matteo
https://it.wikipedia.org/wiki/Matteo_Salvini
https://it.wikipedia.org/wiki/Parlamento
https://it.wikipedia.org/wiki/Che
https://it.wikipedia.org/wiki/Durante
https://it.wikipedia.org/wiki/Emergenza
https://it.wikipedia.org/wiki/Coronavirus
https://it.wikipedia.org/wiki/Ogni
https://it.wikipedia.org/wiki/Caso
https://it.wikipedia.org/wiki/Presente
https://it.wikipedia.org/wiki/Cento
https://it.wikipedia.org/wiki/Casi
https://it.wikipedia.org/wiki/Corso
https://it.wikipedia.org/wiki/Della
https://it.wikipedia.org/wiki/Andiamo
https://it.wikipedia.org/wiki/Se_è
https://it.wikipedia.org/wiki/Vero
https://it.wikipedia.org/wiki/Italiano
https://it.wikipedia.org/wiki/T

# NLTK NLP module

In [182]:
import nltk

sample = acc_data


sentences = nltk.sent_tokenize(sample)
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

entity_names = []
for tree in chunked_sentences:
    # Print results per sentence
    # print extract_entity_names(tree)

    entity_names.extend(extract_entity_names(tree))

# Print all entity names
# print(sorted(names.items(), key=lambda x:x[1], reverse=True))
#print entity_names

# Print unique entity names
print (set(entity_names))

{'M5s Rosalba Cimino', 'Giorgia Meloni', 'Riccardo Merlo', 'OpenPolis', 'Senato', 'Inoltre Salvini', 'Covid19 Il Parlamento', 'Facebook', 'Lega', 'Cimino', 'Salvini Le', 'Matteo Salvini', 'M5s', 'Fratelli', 'Questa', 'Camera', 'Carta', 'Parlamento Facciamo', 'Parlamento', 'Anche', 'Parlamento La', 'Palazzo Madama', 'Maria Bogo Deledda', 'Italia Viva Matteo Renzi', 'Italia', 'Dunque', 'Sostanzialmente Cimino', 'Aule', 'Lega Dal', 'Salvini La', 'M5s Cimino', 'Passiamo', 'Facendo', 'Parlamento Non', 'OpenParlamento', 'Salvini'}
